In [ ]:
# main.ipynb

import os
import pandas as pd
from FileHandler import ask_directory, check_directory, select_excel_files, confirm_action
from SubTables import process_data_file
from MasterTable import combine_and_organize

try:
    # Prompt user to input directories
    input_directory = ask_directory("Enter the address of Input Directory: ")

    # Check if input_directory exists and is valid
    check_directory(input_directory)

    # Select .xlsx or .csv files to process
    selected_files = select_excel_files(input_directory)
    if not selected_files:
        print("Process cancelled. No files selected.")
    else:
        # Ask user for output directory for organized data
        output_directory = ask_directory("Enter the address of Output Directory for Organized Data: ")
        os.makedirs(output_directory, exist_ok=True)

        # Process selected .xlsx or .csv files
        organized_files = []
        for file in selected_files:
            file_path = os.path.join(input_directory, file)

            # Process the file based on its type (.xlsx or .csv)
            excel_file, success = process_data_file(file_path, output_directory)
            if success:
                print(f"Processed '{file}' and saved to '{excel_file}'.")
                organized_files.append(excel_file)
            else:
                print(f"Processing failed for '{file}'.")

        # After processing, ask user if they want to add data to master table
        if confirm_action():
            # Ask user for output directory for master table
            master_table_output = ask_directory("Enter the address of Output Directory for Master Table: ")
            os.makedirs(master_table_output, exist_ok=True)

            # Combine organized data into master table
            master_table_file = os.path.join(master_table_output, "master_table.xlsx")
            combine_and_organize(output_directory, master_table_file)
        else:
            print("Data processing completed. Data was not added to the master table.")

except PermissionError as pe:
    print(f"Permission error occurred: {pe}. Please check permissions.")
except FileNotFoundError as fnf:
    print(f"File not found error: {fnf}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
